# Property Calculations (MDTraj)

In [1]:
# Cusotm imports
from polymer_utils import analysis, filetree, general
from polymer_utils.representation import PolymerDir, PolymerDirManager
from polymer_utils.simulation import SimulationParameters
from polymer_utils.analysis import trajectory, propplot

# General imports
import mdtraj as mdt
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Units
from openmm.unit import nanometer

## Load trajectory and calculate/plot properties

In [2]:
POLY_PATH = Path('Collections/simple_polymers_updated')
SAMPLE_INTERVAL = 5 # interval between consecutively read time step samples (e.g. 1 would read all steps, 2 every other, etc.)

mgr = PolymerDirManager(POLY_PATH)
pdir = mgr.mol_dirs['naturalrubber_solv_water']

if not pdir.chrono_sims:
    raise ValueError(f'No Simulation results found for {pdir.mol_name}')

sim_dir_to_use = pdir.newest_sim_dir
print(sim_dir_to_use)   

Collections/simple_polymers_updated/naturalrubber/naturalrubber_solv_water/MD/04-17-2023_at_15-35-54_PM


In [3]:
# look up and load in relevant simulation data from the chosen directory
sim_params_path = next(sim_dir_to_use.glob('*_parameters.json'))
state_data_path = next(sim_dir_to_use.glob('*_data.csv'))
sim_params = SimulationParameters.from_file(sim_params_path)
state_data = pd.read_csv(state_data_path)

# load simulation frames into an analyzable trajectory
traj_path = next(sim_dir_to_use.glob('*_traj.pdb'))
traj = trajectory.load_traj(traj_path, topo_path=pdir.structure_file, sample_interval=SAMPLE_INTERVAL, remove_solvent=True)
topo = traj.topology


<Popen: returncode: None args: ['xdg-open', PosixPath('Collections/simple_po...>

In [4]:

# save and plot RDF data
rdf_dataframe = trajectory.acquire_rdfs(traj, max_rad=1.0*nanometer)
rdf_dataframe.to_csv(sim_dir_to_use/'rdfs.csv')
rdf_fig, rdf_ax = propplot.plot_rdfs(rdf_dataframe, scale=15.0)
rdf_fig.suptitle(f'Pairwise Radial Distribution Functions - {pdir.mol_name}')
rdf_fig.savefig(sim_dir_to_use/f'RDFs.png', bbox_inches='tight')
plt.close()

# save and plot property data
prop_dataframe = trajectory.acquire_time_props(traj, properties=analysis.DEFAULT_PROPS, time_points=sim_params.time_points[::SAMPLE_INTERVAL]) 
prop_dataframe.to_csv(sim_dir_to_use/'time_series.csv')
prop_fig, prop_ax = propplot.plot_time_props(prop_dataframe, scale=18.0)
prop_fig.suptitle(f'Polymer Shape Properties - {pdir.mol_name}')
prop_fig.savefig(sim_dir_to_use/f'shape_props.png', bbox_inches='tight')
plt.close()

filetree.startfile(sim_dir_to_use)

<Popen: returncode: None args: ['xdg-open', PosixPath('Collections/simple_po...>